# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1025 08:59:19.779036   12067 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1025 08:59:27.356890   12100 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Tue Oct 25 08:59:29 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [114]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/fashionmnist_mnistcnn/epoch_25/experiment_10e-5_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [115]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1025 10:49:44.442048   21490 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1025 10:49:47.877419   21506 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Tue Oct 25 10:49:49 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [116]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1025 10:49:56.147403   21525 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-25-2022 08:49:56 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-25-2022 08:49:56 root         INFO     Starting in cluster mode.
10-25-2022 08:49:56 root         INFO     Starting with experiment replication: 0 with seed: 42
10-25-2022 08:49:56 root         INFO     Starting as Orchestrator
10-25-2022 08:49:56 root         INFO     Starting Orchestrator, initializing resources....
10-25-2022 08:49:56 root         INFO     Loading in cluster configuration file
10-25-2022 08:49:56 root         INFO     Pointing configuration to in cluster configuration.
10-25-2022 08:49:56 root         INFO     Starting cluster manager
10-25-2022 08:49:56 ClusterManager INFO     Spinning up cluster manag

In [128]:
CURRENT_POD_ID="ac17a6e0-2d7b-47f5-9b1c-774c0063d339"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1025 11:14:19.218547   23525 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-25-2022 08:50:02 root         INFO     Loading file config/configuration.fltk.json
10-25-2022 08:50:02 root         INFO     Starting in client mode
10-25-2022 08:50:02 root         INFO     Starting with host=trainjob-ac17a6e0-2d7b-47f5-9b1c-774c0063d339-master-0 and port=23456
10-25-2022 08:50:02 root         INFO     Initializing backend for training process: gloo
10-25-2022 08:50:06 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-25-2022 08:50:07 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-25-2022 08:50:07 root         INFO     Starting Creating client with 0
10-25-2022 08:50:07 Client-0-a

10-25-2022 08:59:07 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,    50] loss: 2.305
10-25-2022 08:59:27 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,   100] loss: 2.307
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 08:59:38 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,     0] loss: 0.046
10-25-2022 08:59:57 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,    50] loss: 2.304
10-25-2022 09:00:18 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,   100] loss: 2.308
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) 

  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 09:09:56 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,     0] loss: 0.046
10-25-2022 09:10:17 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,    50] loss: 2.304
10-25-2022 09:10:37 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,   100] loss: 2.307
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 09:10:47 Client-0-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [25,   

       [  0,  12,   0,   3,   0,   0, 218,   6,  31,   0],
       [  1,  16,   0,   7,   0,   0, 187,   4,  40,   0],
       [  5,   5,   0,   4,   0,   0, 179,   4,  33,   0],
       [  3,  11,   0,   8,   0,   0, 174,   3,  22,   0],
       [  0,   4,   0,  12,   0,   0, 184,   3,  41,   0],
       [  2,  11,   0,   4,   0,   0, 184,   7,  28,   0],
       [  1,   6,   0,   9,   0,   0, 186,   5,  30,   0]])), EpochData(epoch_id=8, num_epochs=26, duration_train=410739, duration_test=3806, loss_train=2.307252593040466, accuracy=9.12, loss=46.14183330535889, class_precision=array([0.08333333, 0.05263158,        nan, 0.08196721,        nan,
              nan, 0.09245868, 0.12195122, 0.09116022,        nan]), class_recall=array([0.00740741, 0.01492537, 0.        , 0.01851852, 0.        ,
       0.        , 0.80995475, 0.0204918 , 0.13983051, 0.        ]), confusion_mat=array([[  2,  17,   0,   6,   0,   0, 205,   2,  38,   0],
       [  2,   4,   0,  10,   0,   0, 209,   1,  42,   0],
  

       0.        , 0.87782805, 0.00409836, 0.13983051, 0.        ]), confusion_mat=array([[  2,  11,   0,   6,   0,   0, 212,   5,  34,   0],
       [  4,   4,   0,  16,   0,   0, 194,   5,  45,   0],
       [  0,  22,   0,   2,   0,   0, 204,   4,  37,   0],
       [  4,   9,   0,   5,   0,   0, 215,   6,  31,   0],
       [  2,   9,   0,  10,   0,   0, 188,   3,  43,   0],
       [  2,   5,   0,   4,   0,   0, 187,   5,  27,   0],
       [  1,   6,   0,   4,   0,   0, 194,   2,  14,   0],
       [  2,   7,   0,   4,   0,   0, 200,   1,  30,   0],
       [  1,   7,   0,   4,   0,   0, 188,   3,  33,   0],
       [  3,   5,   0,  12,   0,   0, 182,   4,  31,   0]])), EpochData(epoch_id=16, num_epochs=26, duration_train=824534, duration_test=4621, loss_train=2.308093423843384, accuracy=8.56, loss=46.137765884399414, class_precision=array([0.23809524, 0.11363636,        nan, 0.13235294,        nan,
              nan, 0.08307534, 0.08823529, 0.07407407,        nan]), class_recall=array([0

              nan, 0.09397715, 0.02380952, 0.09340659,        nan]), class_recall=array([0.01481481, 0.03731343, 0.        , 0.04444444, 0.        ,
       0.        , 0.81900452, 0.00409836, 0.1440678 , 0.        ]), confusion_mat=array([[  4,  12,   0,   9,   0,   0, 204,   4,  37,   0],
       [  2,  10,   0,  13,   0,   0, 205,   6,  32,   0],
       [  3,  14,   0,   3,   0,   0, 192,  10,  47,   0],
       [  2,   9,   0,  12,   0,   0, 207,   3,  37,   0],
       [  0,   5,   0,   3,   0,   0, 192,   7,  48,   0],
       [  3,   4,   0,   8,   0,   0, 181,   3,  31,   0],
       [  3,   5,   0,   1,   0,   0, 181,   4,  27,   0],
       [  1,   5,   0,   8,   0,   0, 185,   1,  44,   0],
       [  0,   6,   0,   5,   0,   0, 189,   2,  34,   0],
       [  3,   4,   0,  11,   0,   0, 190,   2,  27,   0]])), EpochData(epoch_id=24, num_epochs=26, duration_train=1236039, duration_test=3803, loss_train=2.3069765758514404, accuracy=9.32, loss=46.15065050125122, class_precision=array([

In [129]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1025 11:14:49.723269   23588 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-25-2022 08:50:06 root         INFO     Loading file config/configuration.fltk.json
10-25-2022 08:50:06 root         INFO     Starting in client mode
10-25-2022 08:50:06 root         INFO     Starting with host=trainjob-ac17a6e0-2d7b-47f5-9b1c-774c0063d339-master-0 and port=23456
10-25-2022 08:50:06 root         INFO     Initializing backend for training process: gloo
10-25-2022 08:50:06 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-25-2022 08:50:06 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-25-2022 08:50:07 root         INFO

  return F.log_softmax(x)
10-25-2022 08:58:47 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,     0] loss: 0.046
10-25-2022 08:59:07 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,    50] loss: 2.305
10-25-2022 08:59:27 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,   100] loss: 2.307
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 08:59:38 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,     0] loss: 0.046
10-25-2022 08:59:57 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,    50] loss: 2.305
10-25-2022 09:00:18 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,   100] loss: 2.307
/opt/federation

/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 09:09:56 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,     0] loss: 0.046
10-25-2022 09:10:17 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,    50] loss: 2.305
10-25-2022 09:10:37 Client-1-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,   100] loss: 2.308
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

       [  4,  10,   0,   8,   0,   0, 196,   4,  47,   0],
       [  0,  15,   0,   8,   0,   0, 187,   2,  27,   0],
       [  2,   4,   0,   5,   0,   0, 202,   4,  40,   0],
       [  2,   5,   0,   6,   0,   0, 177,   1,  39,   0],
       [  1,   6,   0,   5,   0,   0, 211,   6,  39,   0],
       [  0,  10,   0,   1,   0,   0, 204,   4,  35,   0],
       [  0,   4,   0,   6,   0,   0, 206,   2,  37,   0],
       [  1,   6,   0,  13,   0,   0, 178,   2,  36,   0],
       [  0,   4,   0,   9,   0,   0, 205,   0,  36,   0]])), EpochData(epoch_id=8, num_epochs=26, duration_train=410793, duration_test=4102, loss_train=2.3073764514923094, accuracy=10.08, loss=46.079739570617676, class_precision=array([0.14285714, 0.08108108,        nan, 0.15254237,        nan,
              nan, 0.10502513, 0.1627907 , 0.059375  ,        nan]), class_recall=array([0.00840336, 0.02230483, 0.        , 0.03501946, 0.        ,
       0.        , 0.82283465, 0.02745098, 0.08050847, 0.        ]), confusion_mat

       0.        , 0.81889764, 0.01568627, 0.15677966, 0.        ]), confusion_mat=array([[  3,   5,   0,   2,   0,   0, 192,   6,  30,   0],
       [  1,   7,   0,   9,   0,   0, 188,   3,  61,   0],
       [  1,   9,   0,   4,   0,   1, 191,   4,  29,   0],
       [  0,   6,   0,   6,   0,   0, 196,   6,  43,   0],
       [  0,   8,   0,   4,   0,   0, 182,   2,  34,   0],
       [  3,   8,   0,   7,   0,   0, 207,   8,  35,   0],
       [  2,  11,   0,   4,   0,   0, 208,   6,  23,   0],
       [  0,   4,   0,   9,   0,   0, 196,   4,  42,   0],
       [  0,   6,   0,   2,   0,   0, 189,   2,  37,   0],
       [  2,   5,   0,  14,   0,   0, 197,   4,  32,   0]])), EpochData(epoch_id=16, num_epochs=26, duration_train=824591, duration_test=3908, loss_train=2.307041211128235, accuracy=9.8, loss=46.11237072944641, class_precision=array([0.06666667, 0.06666667,        nan, 0.1       ,        nan,
              nan, 0.10136847, 0.13043478, 0.08211144,        nan]), class_recall=array([0.0

              nan, 0.10346585, 0.34146341, 0.09171598,        nan]), class_recall=array([0.00420168, 0.01486989, 0.        , 0.02723735, 0.        ,
       0.        , 0.7992126 , 0.05490196, 0.13135593, 0.        ]), confusion_mat=array([[  1,   9,   0,   0,   0,   0, 188,   2,  38,   0],
       [  4,   4,   0,  11,   0,   0, 208,   2,  40,   0],
       [  5,  10,   0,   2,   0,   0, 197,   2,  23,   0],
       [  3,  13,   0,   7,   0,   0, 198,   3,  33,   0],
       [  2,   5,   0,   4,   0,   0, 183,   4,  32,   0],
       [  3,   9,   0,   5,   0,   0, 196,   5,  50,   0],
       [  1,  17,   0,   6,   0,   0, 203,   4,  23,   0],
       [  0,   3,   0,   3,   0,   0, 198,  14,  37,   0],
       [  3,   6,   0,   3,   0,   0, 190,   3,  31,   0],
       [  2,  10,   0,   8,   0,   0, 201,   2,  31,   0]])), EpochData(epoch_id=24, num_epochs=26, duration_train=1236096, duration_test=3408, loss_train=2.3079426956176756, accuracy=9.56, loss=46.108718156814575, class_precision=array(

In [130]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1025 11:14:56.780623   23610 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-25-2022 08:50:06 root         INFO     Loading file config/configuration.fltk.json
10-25-2022 08:50:06 root         INFO     Starting in client mode
10-25-2022 08:50:06 root         INFO     Starting with host=trainjob-ac17a6e0-2d7b-47f5-9b1c-774c0063d339-master-0 and port=23456
10-25-2022 08:50:06 root         INFO     Initializing backend for training process: gloo
10-25-2022 08:50:06 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-25-2022 08:50:06 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-25-2022 08:50:07 root         INFO

  return F.log_softmax(x)
10-25-2022 08:58:47 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,     0] loss: 0.046
10-25-2022 08:59:07 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,    50] loss: 2.305
10-25-2022 08:59:27 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [11,   100] loss: 2.307
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 08:59:38 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,     0] loss: 0.046
10-25-2022 08:59:57 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,    50] loss: 2.304
10-25-2022 09:00:18 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [12,   100] loss: 2.308
/opt/federation

/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
10-25-2022 09:09:56 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,     0] loss: 0.046
10-25-2022 09:10:17 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,    50] loss: 2.304
10-25-2022 09:10:37 Client-2-ac17a6e0-2d7b-47f5-9b1c-774c0063d339 INFO     [24,   100] loss: 2.307
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encountered in true_divide
  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
/opt/federation-lab/fltk/nets/mnist_cnn.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

       [  4,   7,   0,  13,   0,   0, 165,   5,  36,   0],
       [  3,  11,   0,   1,   0,   0, 183,   8,  28,   0],
       [  1,   8,   0,   9,   0,   0, 186,   4,  44,   0],
       [  2,  13,   0,   9,   0,   0, 197,   8,  32,   0],
       [  1,   6,   0,   9,   0,   0, 185,   4,  31,   0],
       [  0,  15,   0,   9,   0,   0, 206,   5,  34,   0],
       [  3,   6,   0,   8,   0,   0, 186,   6,  45,   0],
       [  0,   7,   0,   4,   0,   0, 225,   6,  29,   0],
       [  4,   7,   0,   8,   0,   0, 206,   6,  18,   0]])), EpochData(epoch_id=8, num_epochs=26, duration_train=410795, duration_test=4202, loss_train=2.307252593040466, accuracy=10.64, loss=46.07118225097656, class_precision=array([0.        , 0.08695652,        nan, 0.15151515,        nan,
              nan, 0.10778752, 0.02631579, 0.11080332,        nan]), class_recall=array([0.        , 0.02608696, 0.        , 0.03968254, 0.        ,
       0.        , 0.77695167, 0.00393701, 0.14760148, 0.        ]), confusion_mat=a

       0.        , 0.80297398, 0.00787402, 0.14760148, 0.        ]), confusion_mat=array([[  2,   8,   0,   5,   0,   0, 205,   5,  19,   0],
       [  0,   6,   0,  11,   0,   0, 178,   3,  32,   0],
       [  1,  11,   0,   3,   0,   0, 182,   6,  31,   0],
       [  1,   3,   0,   9,   0,   0, 199,   6,  34,   0],
       [  2,  10,   0,   5,   0,   0, 203,   2,  39,   0],
       [  0,   6,   0,  11,   0,   0, 184,   3,  32,   0],
       [  1,   8,   0,   4,   0,   0, 216,   7,  33,   0],
       [  0,   2,   0,   6,   0,   0, 213,   2,  31,   0],
       [  2,   9,   0,   5,   0,   0, 213,   2,  40,   0],
       [  3,   7,   0,  11,   0,   0, 194,   5,  29,   0]])), EpochData(epoch_id=16, num_epochs=26, duration_train=824593, duration_test=4205, loss_train=2.308093423843384, accuracy=10.64, loss=46.07334089279175, class_precision=array([0.        , 0.05555556,        nan, 0.05454545,        nan,
              nan, 0.11026034, 0.        , 0.11965812,        nan]), class_recall=array([0

              nan, 0.10996386, 0.02439024, 0.08839779,        nan]), class_recall=array([0.00819672, 0.02173913, 0.        , 0.02777778, 0.        ,
       0.        , 0.79182156, 0.00393701, 0.11808118, 0.        ]), confusion_mat=array([[  2,   8,   0,   7,   0,   0, 198,   2,  27,   0],
       [  3,   5,   0,   5,   0,   0, 170,   8,  39,   0],
       [  3,  11,   0,   4,   0,   0, 176,   3,  37,   0],
       [  1,   5,   0,   7,   0,   0, 199,   4,  36,   0],
       [  2,  10,   0,   8,   0,   0, 199,   8,  34,   0],
       [  0,   9,   0,   4,   0,   0, 176,   1,  46,   0],
       [  4,  13,   0,   4,   0,   0, 213,   3,  32,   0],
       [  1,   5,   0,   8,   0,   0, 189,   1,  50,   0],
       [  0,   7,   0,   7,   0,   0, 219,   6,  32,   0],
       [  1,   6,   0,  10,   0,   0, 198,   5,  29,   0]])), EpochData(epoch_id=24, num_epochs=26, duration_train=1236093, duration_test=3804, loss_train=2.3069765758514404, accuracy=10.8, loss=46.06731200218201, class_precision=array([

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [131]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1025 11:17:18.596194   23801 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1025 11:17:19.304109   23815 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [132]:
helm uninstall -n test experiment-orchestrator

W1025 11:17:29.586870   23842 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [133]:
helm uninstall extractor -n test

W1025 11:17:36.310075   23857 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [134]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1025 11:18:01.975271   23914 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-06a8547e-765a-4688-a76a-e58e330bdb43" deleted
pytorchjob.kubeflow.org "trainjob-0c0de4ab-dd8d-4589-b096-09802308e755" deleted
pytorchjob.kubeflow.org "trainjob-13ecb4e5-dc83-4e5b-a4cb-fe09b427f500" deleted
pytorchjob.kubeflow.org "trainjob-4505c753-eca9-4590-b94e-ae8be7ffe189" deleted
pytorchjob.kubeflow.org "trainjob-4d34cc70-effa-4fc6-bcdc-bb52ff6a9e6c" deleted
pytorchjob.kubeflow.org "trainjob-73b5efaa-d69b-473d-ad4b-36a99928a3ee" deleted
pytorchjob.kubeflow.org "trainjob-7560abd5-6ee8-41c2-b0c7-860c2b17f6e3" deleted
pytorchjob.kubeflow.org "trainjob-7fc7f5ca-0724-4e02-b7fb-cd3a5df1776b" deleted
pytorchjob.kubeflow.org "trainjob-81e860db-7686-45c8-bf56-fdf2c5f9daa2" deleted
pytorchjob.kubeflow.org "trai